In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # Кроссвалидации при обучении
from sklearn.model_selection import GridSearchCV # Для подбора лучших параметров для обучения решающего дерева
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score, recall_score # Для расчета метрик качества модели
from sklearn.metrics import roc_curve, auc # Для графика рок-кривой
from sklearn.ensemble import RandomForestClassifier # Для создания рандомног леса
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix

events_train.csv - данные о действиях, которые совершают студенты со стэпами

step_id - id стэпа
user_id - анонимизированный id юзера
timestamp - время наступления события в формате unix date
action - событие, возможные значения: 
discovered - пользователь перешел на стэп
viewed - просмотр шага,
started_attempt - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
passed - удачное решение практического шага

submissions_train.csv - данные о времени и статусах сабмитов к практическим заданиям

step_id - id стэпа
timestamp - время отправки решения в формате unix date
submission_status - статус решения
user_id - анонимизированный id юзера

In [1]:
events_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\event_data_train.zip',compression= 'zip')
submissions_train = pd.read_csv('C:\\Users\\Ильнар\\Data science\\submissions_data_train.zip',compression='zip')
events_train.head()
submissions_train.head()

NameError: name 'pd' is not defined

In [ ]:
# Просмотр(Проверка) данных перед предобработкой

# Для дата фрейма events_train
events_train.action.unique()
events_train['date'] = pd.to_datetime(events_train.timestamp, unit = 's')
# events_train.head()
events_train.date.min() # Первый просмотр курса
events_train.date.max() # Последний просмотр
events_train['day'] = events_train.date.dt.date
events_train.head()

# Для дата фрейма submissions_train
submissions_train['date'] = pd.to_datetime(submissions_train.timestamp, unit = 's')
submissions_train['day'] = submissions_train.date.dt.date
submissions_train.head()

In [ ]:
# Просмотр уникальных пользователей по датам
events_train.groupby('day').user_id.nunique()
events_train.groupby('day').user_id.nunique().plot(figsize = (20,10))

In [ ]:
# pivot_table - переворачивает дата фрейм где индексы строк это id пользователя, названия столбцов это значения 
# которые принимает столбец action, values это столбец к которому применяется функция aggfunc

users_events_data = events_train.pivot_table(index = 'user_id', columns = 'action', values = 'step_id', aggfunc = 'count',
                        fill_value = 0).reset_index()
# График распределения
events_train.pivot_table(index = 'user_id', columns = 'action', values = 'step_id', aggfunc = 'count',
                        fill_value = 0).discovered.hist()
users_events_data.head()

In [ ]:
# Просмотр пользователей по количеству равильных ответов и неправильных для submissions_train

user_score = submissions_train.pivot_table(index = 'user_id', columns = 'submission_status', values = 'step_id', 
                                           aggfunc = 'count', fill_value = 0).reset_index()
user_score.head()

In [ ]:
# Рассчитываем период времени для пользователей их периоды отсутствия на курсе
# drop_duplicates - убирает повторяющиеся значения
events_train[['step_id','user_id']].head().drop_duplicates()

In [ ]:
# Для каждого пользователя отфильровали время нахождения на курсе(например если пользователь за 1 день заходил 10 раз
# мы считали только 1 день)
events_train[['user_id','timestamp','day']].drop_duplicates(subset = ['user_id', 'day']).head()

# Для каждого пользователя соберем уникальные дни его нахождения в курсе(вернем результат в виде списка)
events_train[['user_id','timestamp','day']].drop_duplicates(subset = ['user_id', 'day'])\
    .groupby('user_id').timestamp.apply(list).head()

# Для каждого списка считаем сколько времени в виде unix date прошло между ближайшеми днями
events_train[['user_id','timestamp','day']].drop_duplicates(subset = ['user_id', 'day'])\
    .groupby('user_id').timestamp.apply(list)\
    .apply(np.diff).head()

# Вернет массив с массивами разниц времени между ближайшеми днями для каждого пользователя
gap_data= events_train[['user_id','timestamp','day']].drop_duplicates(subset = ['user_id', 'day'])\
    .groupby('user_id').timestamp.apply(list)\
    .apply(np.diff).values

# Все разницы соберем в одну пандосовскую серию для статистического теста
gap_data = pd.Series(np.concatenate(gap_data, axis = 0))
gap_data = gap_data/(24*60*60) # Разница в днях
gap_data

In [ ]:
# Визуализация
# Убираем слишком большие выбросы
gap_data[gap_data<200].hist() # большинство разниц между ближайшеми уникальными днями от 0 до 25 дней

In [ ]:
gap_data.quantile(0.95) # только 5% людей возвращаются после отсутствия на курсе больше 60 дней
gap_data.quantile(0.90)

# Для критического значения возьмем 30 дней между 0.90 и 0.95. Если пользователь не прошел курс до конца и он
# отсутствует больше 30 дней он будет считаться ушедшим из курса
time_porog = 30*24*60*60  # Пороговое значения 30 дней в виде timestamp

In [ ]:
# Последний timestamp - последний день(день когда выгрузили данные и начали анализ, т.е текущее время)
last_day = events_train.timestamp.max() 
time_porog = 30*24*60*60  # Пороговое значения 30 дней в виде timestamp
# относительно этого числа будем считать
# Для каждого пользователя найдем его последний день нахождения на курсе(последний timestamp)
users_data = events_train.groupby('user_id',as_index = False).agg({'timestamp':'max'})\
.rename(columns = {'timestamp':'last_timestamp'})
# Посчитаем для каждого пользователя разницу между текущим днем и его последним посещением курса
# Если после его последнего посещения прошло больше порогового значения и он не закончил курс то он считает ушедшим из курса
users_data['is_gone_user'] = (last_day - users_data.last_timestamp) > time_porog
users_data.head()

In [ ]:
# Добавим к дата фрейму дата фрейм с количесвтом правильных и неправильных ответов по id пользователя
# inner join - берем пересечения user_id в обоих дата фреймах и по ним объединяем.
# если какой-то user_id отсутствует в каком-то дата фрейме мы его выкидываем. Чтобы так не терять пользователей
# функции merge передаем аргумент how = 'outer'
# user_score.head()
# users_data.head()
users_data = users_data.merge(user_score, how='outer')
# users_data.head()
users_data = users_data.fillna(0)
# users_data.head()
# Добавим еще дата фрейм с количеством степов для каждого пользователя
# users_events_data.head()
users_data = users_data.merge(users_events_data, how = 'outer')
users_data.head()

In [ ]:
# Найдем для каждого пользователя число уникальных дней в днях
users_day = events_train.groupby('user_id').day.nunique() # Вернет серию
users_day = users_day.to_frame().reset_index() # превращаем в дата фрейм
users_day.head()

In [ ]:
# Добавим к основному дата фрейму число уникальных дней
users_data.head()
users_data = users_data.merge(users_day, how = 'outer')
# проверяем что мы никого не потеряли
users_data.user_id.count()
events_train.user_id.nunique()

In [ ]:
# Проверим успешно ли закончил пользователь курс
# За границу возьмем 170 степов если меньше значит не закончил
users_data['passed_corse'] = users_data.passed > 170
users_data.head()
users_data.groupby('passed_corse').count().user_id # количество user_id прошедших и не прошедших курс


In [ ]:
# Попробуем предсказать уйдет пользователь или нет за первые несколько дней 

users_data[users_data.passed_corse].day.median() # Медианное значение дней прохождения курса

# Попробуем предсказать уйдет или не уйдет за первые 3 дня
# timestamp первого взаимодействия с курсом для каждого пользователя 
user_min_time = events_train.groupby('user_id',as_index=False)\
            .agg({"timestamp":'min'})\
            .rename({'timestamp':'min_timestamp'},axis=1)  
user_min_time.head()

In [ ]:
users_data = users_data.merge(user_min_time,how='outer')
users_data.head()


In [2]:
# Отбор первых 3 дней для каждого пользователя

events_train['user_time'] = events_train.user_id.map(str) + '_' +  events_train.timestamp.map(str)
events_train.head()
user_learning_time_thresh = user_min_time.user_id.map(str) + '_' + (user_min_time.min_timestamp + time_thresh).map(str)
user_learning_time_thresh.head()
user_min_time['user_learning_time_thresh'] = user_learning_time_thresh
user_min_time.head()
events_train = events_train.merge(user_min_time[['user_id','user_learning_time_thresh']],how='outer')
events_train.head()
events_data_train = events_train[events_train.user_time <= events_train.user_learning_time_thresh]
events_data_train.head()
events_data_train.user_id.nunique()

NameError: name 'events_train' is not defined

In [ ]:
# Для данных submissions_train тоже отбираем для каждого пользователя только первые 3 дня на курсе

submissions_train['users_time'] = submissions_train.user_id.map(str) + '_' + submissions_train.timestamp.map(str)
submissions_train = submissions_train.merge(user_min_time[['user_id', 'user_learning_time_thresh']], how='outer')
submissions_data_train = submissions_train[submissions_train.users_time <= submissions_train.user_learning_time_thresh]
submissions_data_train.groupby('user_id').day.nunique().max()
submissions_data_train.head()


In [ ]:
# Применяем машинное обучение чтобы по первым 3 дня пользователей предсказать уйдет с курса или нет

# Определяем количесвто дней подряд проведенных на курсе с момента первого взаимодействия пользователей в течении 3 дней
X = submissions_data_train.groupby('user_id').day.nunique().to_frame()\
            .reset_index().rename(columns = {'day':'days'})
X.head()

In [ ]:
# Посчитаем сколько степов пытался решить пользователь за первые 3 дня

step_tried = submissions_data_train.groupby('user_id').step_id.nunique().to_frame().reset_index()\
            .rename(columns = {'step_id':'steps_tried'})
step_tried.head()

In [ ]:
X = X.merge(step_tried,on='user_id',how='outer')
X.head()

In [ ]:
# Для каждого пользователя добавим количество пройденных и  непройденных степов за 3 дня
user_score = submissions_data_train.pivot_table(index = 'user_id', columns = 'submission_status', values = 'step_id', 
                                           aggfunc = 'count', fill_value = 0).reset_index()
X = X.merge(user_score,on='user_id',how='outer')
X.head()

In [ ]:
# Соотношение количества правильных ответов ко всем ответам
X['correct_ratio'] = X.correct/(X.correct + X.wrong)
X.head()

In [ ]:
# Количество просмотренных степов для пользователей за 3 дня
X = X.merge(events_data_train.pivot_table(index = 'user_id',
                                         columns = 'action',
                                         values = 'step_id',
                                         aggfunc = 'count',
                                         fill_value = 0).reset_index()[['user_id','viewed']],how = 'outer')
X = X.fillna(0)
X.head()

In [ ]:
X = X.merge(users_data[['user_id','passed_corse','is_gone_user']],how='outer')
X.head()

In [ ]:
# Отбираем для Х пользователей которые или прошли курс или ушли из курса но не прошли его

X = X[~((X.is_gone_user == False) & (X.passed_corse == False))]
X.head()

In [ ]:
# Отбираем X и у для обучения
y = X.passed_corse.map(int)

X = X.drop(['passed_corse','is_gone_user'],axis=1)
X.head()

In [ ]:
# Превращаем user_id в индексы дата фрейма

X = X.set_index(X.user_id)
X.head()
X = X.drop(['user_id'],axis=1)
X.head()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.75)

In [ ]:
params = {'max_depth':range(1,10),'min_samples_split':range(2,20)}
clf = tree.DecisionTreeClassifier(random_state=42)
gr_search = GridSearchCV(clf,params,cv=5)
gr_search.fit(X_train,y_train)

In [ ]:
best_clf = gr_search.best_estimator_
best_clf.score(X_train,y_train)

In [ ]:
best_clf.score(X_test,y_test)

In [ ]:
y_pred = best_clf.predict(X_test)
recall_score(y_test,y_pred)

In [ ]:
pred_y_prob = best_clf.predict_proba(X_test)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, pred_y_prob[:,1])
roc_auc= auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()